## Prep

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

import statsmodels.api as sm
from scipy import stats
import statsmodels.tools as tools
from scipy.stats import bootstrap

from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 100
pd.options.display.max_columns = None

# Set a seed for NumPy
np.random.seed(0)

# Set a seed for Python's built-in random module
random.seed(0)

In [ ]:
def case_when(Z):
    if 'race' in Z and 'site' in Z:
        return 'race_site'
    elif 'race' in Z:
        return 'race'
    elif 'site' in Z:
        return 'site'
    else:
        return None  # Or any default value

In [ ]:
def generate_combinations(Y, X, M, data):
    # Initialize additional_Z based on the 'data' argument
    base_Z = ['age', 'gender', 'Diabetes', 'hdl', 'totchol', 'sbp']  # Common variables
    
    # Define different settings based on the dataset
    if data == "mesa":
        settings = [
            base_Z + ['race', 'site'],
            base_Z + ['race'],
            base_Z + ['site'],
            base_Z
        ]
    elif data == "mesa_bla":
        settings = [
            base_Z + ['site'],
            base_Z
        ]
    elif data == "jhs":
        settings = [base_Z]  # No 'race' or 'site'
    else:
        settings = [base_Z]  # Default setting, can be modified as needed
    
    combinations = []
    for setting in settings:
        for x in X:
            for m in M:
                Y = Y  
                Z = set(setting) - {x, m}  # Adjust Z based on the current setting
                combination = {'Y': Y, 'X': x, 'M': m, 'Z': list(Z)}
                combinations.append(combination)
    return combinations

### Read data

In [ ]:
mesa_std = pd.read_csv('../data_processed/MESA/mesa_std.csv')
mesa_std = mesa_std.dropna()

# recode nutrition and PhysAct
diet_phys_map = {0:1, 1:2, 2:3}
mesa_std['nutrition'] = mesa_std['nutrition'].replace(diet_phys_map)
mesa_std['PhysAct'] = mesa_std['PhysAct'].replace(diet_phys_map)

mesa_bla_std = mesa_std[mesa_std['race'] == 1]

In [ ]:
jhs_std = pd.read_csv('../data_processed/JHS/jhs_std.csv')

jhs_std = jhs_std.dropna()

# recode nutrition and PhysAct
diet_phys_map = {0:1, 1:2, 2:3}
jhs_std['nutrition'] = jhs_std['nutrition'].replace(diet_phys_map)
jhs_std['PhysAct'] = jhs_std['PhysAct'].replace(diet_phys_map)

## Moderation test

### continuous variable standardized
### ordinal categorical and treat as continuous variable
### bootstrap to report CI for the interaction coefficient 

In [ ]:
named_df_list = [('jhs_std', jhs_std)]
data_option = "jhs"

Y_var = 'cvd_10y_noHF'
X_var = ['nSES','nFavFood','nPhysFac', 'nRS']
M_var = ['FamIncome','nutrition', 'PhysAct','currentSmoker','alc']

iteration = 1000

In [ ]:
combinations = generate_combinations(Y_var, X_var, M_var, data_option)

In [ ]:
report_df = pd.DataFrame(columns=['Data','Y','X','moderator', 'other','mean_coef', 'ci_lower','ci_upper' ])


for df_name, df in named_df_list:
    
    for combo in combinations:
        Y = combo['Y']  # The outcome variable
        X = combo['X']  # The treatment or main independent variable
        M = combo['M']  # The moderator variable
        Z = combo['Z']  # The control variables
        
        interaction_coef = []
        
        for i in range(iteration):
            bootstrap_sample = df.sample(n=df.shape[0], replace=True)
            
            # add interaction term
            bootstrap_sample[f'{X}_{M}_interaction'] = bootstrap_sample[X] * bootstrap_sample[M]

            # Define the model formula by combining lists of variables and adding a constant
            model_vars = [X, M] + Z + [f'{X}_{M}_interaction']
            X_model = sm.add_constant(bootstrap_sample[model_vars])

            # Fit the model
            model = sm.Logit(bootstrap_sample[Y], X_model).fit(disp=0)
            
            # store interaction term coef
            interaction_coef.append(model.params[f'{X}_{M}_interaction'])
            
        # store result to report_df
        
        mean_coef = np.round(np.mean(interaction_coef), 5)
        ci_lower = np.round(np.percentile(interaction_coef, q=2.5), 5)
        ci_upper = np.round(np.percentile(interaction_coef, q=97.5), 5)
        
        new_df = pd.DataFrame(
            {'Data': df_name,
             'Y': Y,
             'X': X,
             'moderator': M, 
             'other': case_when(Z),
             'mean_coef':mean_coef,
             'ci_lower': ci_lower,
             'ci_upper':ci_upper 
            }, index=[0])
        
        # append to the complete dataset
        report_df = pd.concat([report_df, new_df], ignore_index=True) 


export_path = f"../results/moderation_noHF_con/{df_name}_moderation.csv"
report_df.to_csv(export_path)


## Mediation Test

### Y = b0 + b1X + bZ + e (Z represents all the other covariates)
### M = b0 + b2X + e
### Y = b0 + b3X + b4M + bZ + e

In [ ]:
# define the data

named_df_list = [('jhs_std', jhs_std)]

In [ ]:
# Define the variables

Y = 'cvd_10y_noHF'
X = ['nSES', 'nFavFood', 'nPhysFac', 'nRS']
M = ['FamIncome', 'nutrition', 'PhysAct', 'currentSmoker', 'alc']

data_option = "jhs" 
combinations = generate_combinations(Y, X, M, data_option)


In [ ]:
### df to store result
report_df = pd.DataFrame(columns = ['Data','X','M','other',
                                    'Effect','Estimate','95_CI_Lower','95_CI_Upper'])

for df_name, df in named_df_list:
    
    for combo in combinations:
        Y = combo['Y']  # The outcome variable
        X = combo['X']  # The treatment or main independent variable
        M = combo['M']  # The mediator variable
        Z = combo['Z']  # The control variables
    
        ### list to store bootstrapping result
        num_bootstrap_samples = 1000   ######
        bootstrap_result_b1 = []
        bootstrap_result_b2 = []
        bootstrap_result_b3 = []
        bootstrap_result_b4 = []
    
        for _ in range(num_bootstrap_samples):

            if _ % 100 == 0 and _ != 0: 
                print(_)

            # Draw bootstrap sample
            bootstrap_sample = df.sample(frac=1, replace=True)

            # Y ~ X + Z
            model_1 = sm.Logit(bootstrap_sample[Y], sm.add_constant(bootstrap_sample[[X]+Z])).fit(disp=0)  ## +Z
            bootstrap_result_b1.append(model_1.params[X])

            # M ~ X
            model_2 = sm.OLS(bootstrap_sample[M], sm.add_constant(bootstrap_sample[X])).fit()
            bootstrap_result_b2.append(model_2.params[X])

            # Y ~ X + M +Z
            model_3 = sm.Logit(bootstrap_sample[Y], sm.add_constant(bootstrap_sample[[X]+[M]+Z])).fit(disp=0)
            bootstrap_result_b3.append(model_3.params[X])
            bootstrap_result_b4.append(model_3.params[M])


        ### report result

        print(f"Mediation results using: {df_name}\n")
        print(f"Neighborhood exposure variable: {X}\n")
        print(f"Individual level potential mediator tested: {M}\n")

        # result for a each dataset and each variable combo
        new_df = pd.DataFrame(
            {'Data': [df_name, df_name,df_name,df_name],
             'X': [X,X,X,X],
             'M': [M,M,M,M],
             'other': case_when(Z),
             'Effect': ['ACME', 'ADE', 'Total Effect', 'Prop. Mediated'], 
             'Estimate': [round((np.mean(bootstrap_result_b2)*np.mean(bootstrap_result_b3)), 5),
                          round(np.mean(bootstrap_result_b3), 5),
                          round(np.mean(bootstrap_result_b1), 5),
                          round(np.mean(bootstrap_result_b2)*np.mean(bootstrap_result_b3)/np.mean(bootstrap_result_b1), 5)],
             '95_CI_Lower': [round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3), 2.5),5),
                             round(np.percentile(bootstrap_result_b3, 2.5),5),
                             round(np.percentile(bootstrap_result_b1, 2.5),5),
                             round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3)/np.array(bootstrap_result_b1), 2.5),5)],
             '95_CI_Upper': [round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3), 97.5),5),
                             round(np.percentile(bootstrap_result_b3, 97.5),5),
                             round(np.percentile(bootstrap_result_b1, 97.5),5),
                             round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3)/np.array(bootstrap_result_b1), 97.5),5)]
            })

        # append to the complete dataset
        report_df = pd.concat([report_df, new_df], ignore_index=True)

        # print(report_df)

        print("\n" + "="*50 + "\n")  
        
export_path = f"../results/mediation_noHF_con/{df_name}_mediation.csv"
report_df.to_csv(export_path)